## Prepare imports

In [5]:
from datasets import get_datasets, synsetid_to_cate
from args import get_parser
from pprint import pprint
# from metrics.evaluation_metrics import EMD_CD
# from metrics.evaluation_metrics import jsd_between_point_cloud_sets as JSD
# from metrics.evaluation_metrics import compute_all_metrics
from collections import defaultdict
from models.networks import PointFlow
import os
import torch
import numpy as np
import torch.nn as nn
import pyvista as pv



################################################################################
# Import model and additional stuff
################################################################################

from omegaconf import OmegaConf
from model_wrapper import TopologicalModelEncoderScaled
from load_models import load_encoder, load_vae


################################################################################
################################################################################


def get_test_loader(args):
    _, te_dataset = get_datasets(args)
    if args.resume_dataset_mean is not None and args.resume_dataset_std is not None:
        mean = np.load(args.resume_dataset_mean)
        std = np.load(args.resume_dataset_std)
        te_dataset.renormalize(mean, std)
    loader = torch.utils.data.DataLoader(
        dataset=te_dataset,
        batch_size=args.batch_size,
        shuffle=False,
        num_workers=0,
        pin_memory=True,
        drop_last=False,
    )
    return loader



In [6]:
p = get_parser()
args, unknown = p.parse_known_args()
args.cates = ["airplane"]

In [7]:
# loader = get_test_loader(args)

# for data in loader:
#     idx_b, tr_pc, te_pc = data["idx"], data["train_points"], data["test_points"]
#     break

# pl = pv.Plotter(shape=(8,8), window_size=[1600, 1600],border=False,polygon_smoothing=True)

# for row in range(8):
#     for col in range(4):
#         points = te_pc[row*col + col].reshape(-1, 3).detach().cpu().numpy()
#         pl.subplot(row, col)
#         actor = pl.add_points(
#             points,
#             style="points",
#             emissive=False,
#             show_scalar_bar=False,
#             render_points_as_spheres=True,
#             scalars=points[:, 2],
#             point_size=5,
#             ambient=0.2, 
#             diffuse=0.8, 
#             specular=0.8,
#             specular_power=40, 
#             smooth_shading=True
#         )


# pl.background_color = "w"
# pl.link_views()
# pl.camera_position = "yz"
# pos = pl.camera.position
# pl.camera.position = (pos[0],pos[1],pos[2]+3)
# pl.camera.azimuth = -45
# pl.camera.elevation = 10
# # create a top down light
# light = pv.Light(position=(0, 0, 3), positional=True,
#                 cone_angle=50, exponent=20, intensity=.2)
# pl.add_light(light)
# pl.camera.zoom(1.3)
# pl.show()

In [8]:

from model_wrapper import DEVICE
from models.vae import VanillaVAE
from models.vae import BaseModel as BaseVAE

from models.encoder import BaseModel as EctEncoder

from metrics.metrics import get_mse_metrics
from metrics.accuracies import compute_mse_accuracies
from metrics.loss import compute_mse_loss_fn

from layers.ect import EctLayer, EctConfig
from layers.directions import generate_directions

from lightning.pytorch.utilities.model_summary import ModelSummary
encoder_config = OmegaConf.load(
    f"./configs/config_encoder_shapenet_{args.cates[0]}.yaml"
)

layer = EctLayer(
        EctConfig(
            num_thetas=encoder_config.layer.ect_size,
            bump_steps=encoder_config.layer.ect_size,
            normalized=True,
            device=DEVICE,
            r=7,
        ),
        v=generate_directions(
            encoder_config.layer.ect_size, encoder_config.layer.dim, DEVICE
        ),
    )
    # Load the encoder

encoder_model = EctEncoder.load_from_checkpoint(
    checkpoint_path=f"./trained_models/ectencoder_shapenet_airplane.ckpt",
    layer=layer,
    ect_size=encoder_config.layer.ect_size,
    hidden_size=encoder_config.model.hidden_size,
    num_pts=encoder_config.model.num_pts,
    num_dims=encoder_config.model.num_dims,
    learning_rate=encoder_config.model.learning_rate,
).to(DEVICE)

print(ModelSummary(encoder_model))
model = TopologicalModelEncoderScaled(encoder_model)


C:\Users\ernst\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\lightning\pytorch\utilities\migration\utils.py:56: The loaded checkpoint was produced with Lightning v2.3.3, which is newer than your current Lightning version: v2.2.3


  | Name                | Type             | Params
---------------------------------------------------------
0 | layer               | EctLayer         | 0     
1 | training_accuracy   | MeanSquaredError | 0     
2 | validation_accuracy | MeanSquaredError | 0     
3 | test_accuracy       | MeanSquaredError | 0     
4 | loss_fn             | MSELoss          | 0     
5 | model               | Sequential       | 39.9 M
---------------------------------------------------------
39.9 M    Trainable params
0         Non-trainable params
39.9 M    Total params
159.433   Total estimated model params size (MB)


In [9]:
# TODO: make this memory efficient
if "all" in args.cates:
    cates = list(synsetid_to_cate.values())
else:
    cates = args.cates
all_results = {}
cate_to_len = {}
for cate in cates:
    args.cates = [cate]
    loader = get_test_loader(args)

    all_sample = []
    all_ref = []
    for data in loader:
        idx_b, tr_pc, te_pc = data["idx"], data["train_points"], data["test_points"]

        print(te_pc.norm(dim=-1).max(dim=-1)[0])
        # ########################
        # ## Insert
        # ########################

        # # For comparison, we scale the output for test point cloud to have
        # # unit radius. The radius of the recon pc is scaled with the same
        # # value to make sure relative distances are preserved.
        # # @ErnstRoell

        # te_pc_means = te_pc.mean(axis=1, keepdim=True)
        # te_pc = te_pc - te_pc_means
        # tr_pc = tr_pc - te_pc_means

        # te_pc_norms = torch.norm(te_pc, dim=2).max(axis=1)[0].reshape(-1, 1, 1)

        # te_pc = te_pc / te_pc_norms
        # tr_pc = tr_pc / te_pc_norms

        # ########################
        # ## End insert
        # ########################

        te_pc = te_pc.cuda() if args.gpu is None else te_pc.cuda(args.gpu)
        tr_pc = tr_pc.cuda() if args.gpu is None else tr_pc.cuda(args.gpu)
        B, N = te_pc.size(0), te_pc.size(1)
        out_pc = model.reconstruct(tr_pc, num_points=N)

        m, s = data["mean"].float(), data["std"].float()
        m = m.cuda() if args.gpu is None else m.cuda(args.gpu)
        s = s.cuda() if args.gpu is None else s.cuda(args.gpu)
        out_pc = out_pc * s + m
        te_pc = te_pc * s + m

        # ########################
        # ## Insert
        # ########################

        # # For comparison, we scale the output for test point cloud to have
        # # unit radius. The radius of the recon pc is scaled with the same
        # # value to make sure relative distances are preserved.
        # # @ErnstRoell
        # te_pc_means = te_pc.mean(axis=1, keepdim=True)
        # te_pc = te_pc - te_pc_means
        # out_pc = out_pc - te_pc_means

        # te_pc_norms = torch.norm(te_pc, dim=2).max(axis=1)[0].reshape(-1, 1, 1)
        # te_pc = te_pc / te_pc_norms
        # out_pc = out_pc / te_pc_norms

        # ########################
        # ## End insert
        # ########################

        all_sample.append(out_pc)
        all_ref.append(te_pc)

    sample_pcs = torch.cat(all_sample, dim=0)
    ref_pcs = torch.cat(all_ref, dim=0)
    cate_to_len[cate] = int(sample_pcs.size(0))


Total number of data:2832
Min number of points: (train)2048 (test)2048
Total number of data:405
Min number of points: (train)2048 (test)2048
tensor([3.6876, 3.0290, 3.7320, 3.2906, 3.2039, 3.0738, 3.4907, 3.0860, 3.5382,
        3.7431, 3.0828, 3.0791, 3.1836, 3.2257, 3.1690, 3.2280, 3.2210, 3.5559,
        3.0596, 3.0620, 3.0451, 3.1143, 2.9295, 3.6094, 3.5135, 3.0410, 3.1097,
        3.0283, 3.8888, 3.6260, 3.8200, 3.0624, 3.2083, 3.2098, 3.4987, 3.3411,
        3.4285, 3.1004, 3.1818, 3.3128, 3.6336, 3.0471, 3.9944, 3.1822, 3.3031,
        3.0645, 3.6572, 3.3156, 3.1552, 3.3990])
tensor([3.0464, 3.1612, 3.1800, 3.1124, 3.6533, 3.2761, 4.0000, 3.2018, 3.2360,
        3.6707, 3.2478, 3.2785, 3.0361, 3.1183, 3.1406, 3.6458, 3.1537, 3.4353,
        3.1710, 3.1846, 3.2359, 3.1036, 3.2891, 3.2714, 3.3771, 3.0581, 3.5683,
        3.2050, 3.2350, 3.5732, 3.0564, 3.3012, 3.3059, 3.5761, 3.2734, 3.1427,
        3.1477, 3.1944, 3.6756, 3.1573, 3.8811, 3.2083, 3.8201, 3.0873, 3.1570,
        3.

# Reference PointClouds

In [10]:
from lib2to3 import refactor
import matplotlib.pyplot as plt 


fig,axes = plt.subplots(nrows=3,ncols=8,figsize=(16,6))
fig.tight_layout()
for axis, ref, pred in zip(axes.T,ref_pcs,sample_pcs):
    pts_ref = ref.view(-1,3)[:,[0,2]].cpu().detach()
    ax = axis[0]
    ax.scatter(pts_ref[:,0],pts_ref[:,1],s=.1)
    ax.axis("off")

    ax = axis[1]
    pts_pred = pred.view(-1,3)[:,[0,2]].cpu().detach()
    ax.scatter(pts_pred[:,0],pts_pred[:,1],s=.1)
    ax.axis("off")
    ax = axis[2]
    pts = torch.vstack([pts_pred,pts_ref])
    ax.scatter(pts[:,0],pts[:,1],s=.1)
    ax.axis("off")

plt.savefig("recon.png")

In [11]:
ref_pcs[:4].norm(dim=-1).max()


tensor(0.4085, device='cuda:0')

In [12]:
sample_pcs[:4].norm(dim=-1).max()


tensor(0.4074, device='cuda:0', grad_fn=<MaxBackward1>)

In [13]:
from torch_geometric.data import Data, Batch 

batch = Batch.from_data_list([Data(x=pts.view(-1, 3)) for pts in ref_pcs[:4]])

batch = batch.to(DEVICE)
ect = layer(batch, batch.batch)

In [14]:
import matplotlib.pyplot as plt 
plt.imshow(ect[3].cpu().squeeze().detach().numpy())
plt.savefig("test_ect.png")

In [15]:
jnt = torch.cat([sample_pcs,ref_pcs],axis=1)

jnt.shape

torch.Size([405, 4096, 3])

In [27]:
pl = pv.Plotter(shape=(3,10), window_size=[1600, 600],border=False,polygon_smoothing=True)
offset=260

for col in range(10):
    points = ref_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(0, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )
    points = sample_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(1, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )
    points = jnt[col+offset].reshape(-1, 3).detach().cpu().numpy()
    pl.subplot(2, col)
    actor = pl.add_points(
        points,
        style="points",
        emissive=False,
        show_scalar_bar=False,
        render_points_as_spheres=True,
        scalars=points[:, 2],
        point_size=2,
        ambient=0.2, 
        diffuse=0.8, 
        specular=0.8,
        specular_power=40, 
        smooth_shading=True
    )


pl.background_color = "w"
pl.link_views()
pl.camera_position = "yz"
pos = pl.camera.position
print(pos)
# pl.camera.position = (pos[0],pos[1]+3,pos[2])
pl.camera.position = (0,2,0)
pl.camera.azimuth = 0
pl.camera.elevation = 0
# create a top down light
light = pv.Light(position=(0, 0, 0), positional=True,
                cone_angle=50, exponent=20, intensity=.2)
pl.add_light(light)
pl.camera.zoom(1)
# pl.screenshot(f"./img_encoder/reconstructed_pointcloud{offset}.png",transparent_background=True,scale=2)
pl.show()
# pl.clear()



(2.356682691205888, 0.05244923755526543, 0.017630770802497864)


Widget(value='<iframe src="http://localhost:65223/index.html?ui=P_0x1bef9aa98a0_6&reconnect=auto" class="pyvis…

In [28]:
pl = pv.Plotter(shape=(3,10), window_size=[1600, 600],border=False,polygon_smoothing=True)
for offset in range(0,400,10):
    for col in range(10):
        points = ref_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
        pl.subplot(0, col)
        actor = pl.add_points(
            points,
            style="points",
            emissive=False,
            show_scalar_bar=False,
            render_points_as_spheres=True,
            scalars=points[:, 2],
            point_size=5,
            ambient=0.2, 
            diffuse=0.8, 
            specular=0.8,
            specular_power=40, 
            smooth_shading=True
        )
        points = sample_pcs[col+offset].reshape(-1, 3).detach().cpu().numpy()
        pl.subplot(1, col)
        actor = pl.add_points(
            points,
            style="points",
            emissive=False,
            show_scalar_bar=False,
            render_points_as_spheres=True,
            scalars=points[:, 2],
            point_size=5,
            ambient=0.2, 
            diffuse=0.8, 
            specular=0.8,
            specular_power=40, 
            smooth_shading=True
        )
        points = jnt[col+offset].reshape(-1, 3).detach().cpu().numpy()
        pl.subplot(2, col)
        actor = pl.add_points(
            points,
            style="points",
            emissive=False,
            show_scalar_bar=False,
            render_points_as_spheres=True,
            scalars=points[:, 2],
            point_size=5,
            ambient=0.2, 
            diffuse=0.8, 
            specular=0.8,
            specular_power=40, 
            smooth_shading=True
        )


    pl.background_color = "w"
    pl.link_views()
    pl.camera_position = "yz"
    pos = pl.camera.position
    pl.camera.position = (pos[0],pos[1],pos[2]+3)
    pl.camera.azimuth = -45
    pl.camera.elevation = 10
    # create a top down light
    light = pv.Light(position=(0, 0, 3), positional=True,
                    cone_angle=50, exponent=20, intensity=.2)
    pl.add_light(light)
    pl.camera.zoom(1.3)
    pl.screenshot(f"./pointcloud_{offset}.png",transparent_background=True,scale=2)
    pl.clear_actors()

In [18]:
# pl = pv.Plotter(shape=(8,8), window_size=[1600, 1600],border=False,polygon_smoothing=True)

# for row in range(8):
#     for col in range(8):
#         points = jnt[row*col + col].reshape(-1, 3).detach().cpu().numpy()
#         pl.subplot(row, col)
#         actor = pl.add_points(
#             points,
#             style="points",
#             emissive=False,
#             show_scalar_bar=False,
#             render_points_as_spheres=True,
#             scalars=points[:, 2],
#             point_size=5,
#             ambient=0.2, 
#             diffuse=0.8, 
#             specular=0.8,
#             specular_power=40, 
#             smooth_shading=True
#         )


# pl.background_color = "w"
# pl.link_views()
# pl.camera_position = "yz"
# pos = pl.camera.position
# pl.camera.position = (pos[0],pos[1],pos[2]+3)
# pl.camera.azimuth = -45
# pl.camera.elevation = 10
# # create a top down light
# light = pv.Light(position=(0, 0, 3), positional=True,
#                 cone_angle=50, exponent=20, intensity=.2)
# pl.add_light(light)
# pl.camera.zoom(1.3)
# pl.show()

# Reconstructed Point Clouds

In [19]:
# pl = pv.Plotter(shape=(8,8), window_size=[1600, 1600],border=False,polygon_smoothing=True)

# offset = 200

# for row in range(8):
#     for col in range(8):
#         points = sample_pcs[offset + row*col + col].reshape(-1, 3).detach().cpu().numpy()
#         pl.subplot(row, col)
#         actor = pl.add_points(
#             points,
#             style="points",
#             emissive=False,
#             show_scalar_bar=False,
#             render_points_as_spheres=True,
#             scalars=points[:, 2],
#             point_size=5,
#             ambient=0.2, 
#             diffuse=0.8, 
#             specular=0.8,
#             specular_power=40, 
#             smooth_shading=True
#         )


# pl.background_color = "w"
# pl.link_views()
# pl.camera_position = "yz"
# pos = pl.camera.position
# pl.camera.position = (pos[0],pos[1],pos[2]+3)
# pl.camera.azimuth = -45
# pl.camera.elevation = 10
# # create a top down light
# light = pv.Light(position=(0, 0, 3), positional=True,
#                 cone_angle=50, exponent=20, intensity=.2)
# pl.add_light(light)
# pl.camera.zoom(1.3)
# pl.show()

In [20]:
# samples_norm = sample_pcs.max(axis=-1)[0]
# ref_norm = ref_pcs.max(axis=-1)[0]
# print(ref_norm.shape)
# print(samples_norm.shape)


# print(ref_norm.max(axis=-1)[0])
# print(samples_norm.max(axis=-1)[0])

In [21]:
# import torch
# cd = torch.tensor([1.0181e-04, 9.2737e-05, 1.1672e-04, 8.7424e-05, 9.5222e-05, 1.0118e-04,
#         1.0156e-04, 9.1587e-05, 1.2758e-04, 9.4513e-05, 1.0750e-04, 1.1009e-04,
#         1.1467e-04, 1.0125e-04, 1.0361e-04, 8.9797e-05, 1.8066e-04, 1.2059e-04,
#         8.7267e-05, 7.8763e-05, 1.3131e-04, 9.7592e-05, 2.0223e-04, 1.0888e-04,
#         1.0560e-04, 9.4342e-05, 9.4231e-05, 1.0776e-04, 9.3075e-05, 1.2545e-04,
#         1.0671e-04, 9.5280e-05, 1.0603e-04, 7.9804e-05, 9.8851e-05, 1.2386e-04,
#         1.0586e-04, 7.9189e-05, 1.0687e-04, 9.2862e-05, 1.2424e-04, 1.3726e-04,
#         1.0802e-04, 1.1281e-04, 3.2815e-04, 8.8216e-05, 1.0859e-04, 1.0685e-04,
#         1.0818e-04, 1.0067e-04, 8.9873e-05, 1.1124e-04, 9.4109e-05, 1.0042e-04,
#         1.0662e-04, 1.0232e-04, 9.2823e-05, 1.1845e-04, 1.4984e-04, 1.1666e-04,
#         1.0468e-04, 1.0460e-04, 9.3174e-05, 9.2733e-05, 1.1139e-04, 9.4802e-05,
#         1.1481e-04, 1.3138e-04, 1.0916e-04, 1.0311e-04, 1.1614e-04, 8.5114e-05,
#         2.0449e-04, 1.1781e-04, 9.3904e-05, 1.0662e-04, 1.0552e-04, 1.4454e-04,
#         1.1765e-04, 1.0696e-04, 1.1075e-04, 1.0030e-04, 1.0653e-04, 1.2786e-04,
#         9.2552e-05, 9.6168e-05, 8.6787e-05, 9.6798e-05, 1.0762e-04, 1.1090e-04,
#         1.3116e-04, 1.0775e-04, 1.2752e-04, 8.6095e-05, 9.2154e-05, 1.0390e-04,
#         1.0100e-04, 1.0898e-04, 1.0161e-04, 1.2156e-04, 1.0097e-04, 1.1751e-04,
#         1.1965e-04, 1.0361e-04, 1.0291e-04, 1.0514e-04, 1.1809e-04, 8.4403e-05,
#         1.2753e-04, 9.2619e-05, 1.3546e-04, 1.2384e-04, 1.3618e-04, 8.4599e-05,
#         1.0786e-04, 8.2836e-05, 8.9971e-05, 1.1162e-04, 9.5723e-05, 1.1213e-04,
#         8.3462e-05, 9.7139e-05, 9.2273e-05, 9.8969e-05, 9.4978e-05, 5.5579e-04,
#         1.3485e-04, 1.4018e-04, 1.0923e-04, 1.0253e-04, 8.8266e-05, 1.0662e-04,
#         1.2319e-04, 9.1613e-05, 1.1658e-04, 1.1209e-04, 1.8744e-04, 9.3917e-05,
#         1.0221e-04, 9.2784e-05, 9.5928e-05, 1.2863e-04, 1.1686e-04, 1.1306e-04,
#         1.2762e-04, 8.5150e-05, 1.0812e-04, 1.0274e-04, 9.2561e-05, 1.0102e-04,
#         8.9827e-05, 1.5410e-04, 1.0193e-04, 9.7561e-05, 1.0102e-04, 1.1089e-04,
#         9.3220e-05, 1.7981e-04, 1.1021e-04, 3.6641e-04, 9.3402e-05, 4.2095e-04,
#         1.0797e-04, 1.1395e-04, 9.5111e-05, 1.0971e-04, 1.2419e-04, 9.0080e-05,
#         1.1011e-04, 4.9721e-04, 1.0915e-04, 8.9056e-05, 1.1601e-04, 8.8574e-05,
#         1.3060e-04, 1.0269e-04, 1.2925e-04, 1.0964e-04, 3.1983e-04, 9.1368e-05,
#         9.2768e-05, 1.1090e-04, 1.0847e-04, 1.1715e-04, 4.6638e-04, 1.1166e-04,
#         1.1245e-04, 1.3605e-04, 8.4502e-05, 9.5060e-05, 1.0145e-04, 1.0488e-04,
#         1.6524e-04, 1.4249e-04, 9.2200e-05, 1.3979e-04, 1.0327e-04, 1.1784e-04,
#         1.1485e-04, 1.0958e-04, 2.3794e-04, 1.0003e-04, 9.6970e-05, 8.9134e-05,
#         1.2233e-04, 9.5082e-05, 9.0009e-05, 7.7066e-05, 8.4020e-05, 1.0234e-04,
#         1.1630e-04, 1.0094e-04, 1.2238e-04, 1.0223e-04, 1.0398e-04, 1.0168e-04,
#         1.0655e-04, 1.0318e-04, 9.6312e-05, 9.8703e-05, 8.7142e-05, 9.5998e-05,
#         9.3867e-05, 1.0656e-04, 1.0113e-04, 1.0113e-04, 1.3803e-04, 1.0948e-04,
#         1.0018e-04, 1.0453e-04, 1.1903e-04, 1.1636e-04, 1.2289e-04, 1.1507e-04,
#         1.0982e-04, 1.1285e-04, 8.7514e-05, 1.1843e-04, 1.2647e-04, 9.5243e-05,
#         1.1106e-04, 1.0292e-04, 1.4243e-04, 1.0061e-04, 9.8037e-05, 1.4383e-04,
#         9.1886e-05, 1.0827e-04, 2.0124e-04, 9.5268e-05, 8.5165e-05, 9.5133e-05,
#         9.9468e-05, 2.0675e-04, 9.4362e-05, 9.6885e-05, 1.9738e-04, 9.7040e-05,
#         9.8617e-05, 1.6185e-04, 1.2655e-04, 1.0479e-04, 9.2674e-05, 9.2161e-05,
#         1.0796e-04, 8.9307e-05, 7.4180e-04, 1.0392e-04, 9.6216e-05, 1.0088e-04,
#         1.2107e-04, 2.0089e-04, 9.6079e-05, 1.0065e-04, 4.5174e-04, 1.3691e-04,
#         1.0046e-04, 1.2051e-04, 8.8847e-05, 9.3673e-05, 9.8204e-05, 8.9552e-05,
#         1.1244e-04, 1.0779e-04, 9.2790e-05, 1.0259e-04, 1.1064e-04, 1.0114e-04,
#         1.5070e-04, 1.1024e-04, 9.2154e-05, 1.2067e-04, 1.0509e-04, 9.5591e-05,
#         1.2147e-04, 1.0517e-04, 1.1466e-04, 9.3077e-05, 1.0594e-04, 8.5590e-05,
#         1.8096e-04, 1.0871e-04, 4.8627e-04, 9.8446e-05, 1.6843e-04, 9.8439e-05,
#         1.0823e-04, 1.0462e-04, 1.0150e-04, 1.0188e-04, 1.0207e-04, 1.0639e-04,
#         9.6179e-05, 1.6164e-04, 1.1692e-04, 1.1239e-04, 1.0129e-04, 9.7906e-05,
#         9.9086e-05, 1.2597e-04, 9.2412e-05, 1.1048e-04, 1.5363e-04, 1.3546e-04,
#         9.0905e-05, 1.0546e-04, 1.0373e-04, 9.9698e-05, 1.2578e-04, 1.1118e-04,
#         8.5913e-05, 1.4649e-04, 1.4647e-04, 1.1804e-04, 1.1315e-04, 9.7830e-05,
#         1.4924e-04, 1.1718e-04, 1.4573e-04, 1.0846e-04, 9.4926e-05, 9.6743e-05,
#         8.7338e-05, 9.2540e-05, 1.0258e-04, 1.3488e-04, 1.1887e-04, 7.9505e-05,
#         9.7693e-05, 1.3626e-04, 9.4275e-05, 1.2602e-04, 1.3206e-04, 9.2633e-05,
#         1.7793e-04, 1.2600e-04, 1.0479e-04, 9.1649e-05, 9.8827e-05, 1.1586e-04,
#         1.0939e-04, 1.0014e-04, 1.2602e-04, 1.0437e-04, 9.2961e-05, 1.2004e-04,
#         1.0172e-04, 1.0398e-04, 1.0912e-04, 1.0809e-04, 1.1036e-04, 1.0016e-04,
#         1.1612e-04, 1.1770e-04, 1.2186e-04, 1.0037e-04, 9.3744e-05, 1.0012e-04,
#         1.0324e-04, 9.3971e-05, 1.6649e-04, 1.3841e-04, 7.8760e-05, 1.1158e-04,
#         9.4522e-05, 9.8713e-05, 7.1881e-04, 9.0226e-05, 9.2053e-05, 1.0724e-04,
#         1.3557e-04, 1.2918e-04, 8.7340e-05, 9.2334e-05, 8.7485e-05, 9.5231e-05,
#         9.2954e-05, 1.0898e-04, 1.3990e-04, 9.0515e-05, 1.2379e-04, 3.8274e-04,
#         1.0265e-04, 1.2823e-04, 1.0901e-04])


# import matplotlib.pyplot as plt 


# # plt.switch_backend('Qt5Agg')

# # plt.hist(cd, bins='auto')  # arguments are passed to np.histogram
# # plt.title("Histogram with 'auto' bins")
# # plt.show()

# # cd.mean()
